In [1]:
import cv2
import hand_track as ht
import time
import numpy as np
import math
import mediapipe as mp
import autopy

In [2]:
timeP = 0        # past time

px, py = 0, 0   # past coordinate of x
ctx, cty = 0, 0   # present coordinate of center of Xs

red_frame = 130    # parameter by which the frame is reduced

smooth = 8    # smoothening parameter
loc_px, loc_py = 0, 0     # past location of cursor 
loc_cx, loc_cy = 0, 0     # present location of cursor

cap = cv2.VideoCapture(0)

cam_w, cam_h = 640, 480    # camera parameters
scr_w, scr_h = autopy.screen.size()    # screen parameters

cap.set(3, 640)
cap.set(4, 480)

tracker = ht.hand_tracker(max_num_hands=1, detection_conf=0.6, tracking_conf=0.7)

In [3]:
while True:
    ret, frame = cap.read()
    frame = tracker.detection(frame)
    lm_coord, bbox = tracker.position(frame)
    
    # tip of index and middle fingers
    if len(lm_coord)!=0:
        idx_x, idx_y = lm_coord[8][1], lm_coord[8][2]
        mid_x, mid_y = lm_coord[12][1], lm_coord[12][2]
    
        fingers = tracker.fingers_pos()
    
       # for reaching the areas of the screen where hand is not prperly detected, set a reduced frame 
       #size i.e. reaching the corner of the reduced frame, cursor reaches the corner of actual screen
        cv2.rectangle(frame, (red_frame, red_frame), (cam_w-red_frame, cam_h-red_frame), (255,0,255), 3)
    
        # index finger is moving i.e. moving the mouse
        if fingers[1]==1 and fingers[2]==0 :
            x0 = np.interp(idx_x, (red_frame, cam_w-red_frame), (0, scr_w))
            y0 = np.interp(idx_y, (red_frame, cam_h-red_frame), (0, scr_h))
            
            # to stop shaking of the cursor, smoothen the cursor movement
            loc_cx = loc_px + (x0 - loc_px)/smooth
            loc_cy = loc_py + (y0 - loc_py)/smooth
            
            autopy.mouse.move(scr_w-loc_cx, loc_cy)
            cv2.circle(frame, (idx_x, idx_y), 10, (255,255,255), cv2.FILLED)
            
            loc_px, loc_py = loc_cx, loc_cy
            
        # index and middle finger moving i.e. clicking with the mouse
        if fingers[1]==1 and fingers[2]==1 :
            length, idx2mid_line = tracker.distance(8, 12)
            if length < 10 :
                autopy.mouse.click()
                cv2.circle(frame, (idx2mid_line[4], idx2mid_line[5]), 10, (255,0,255), cv2.FILLED)     
        
    timeC = time.time()
    fps = 1/(timeC-timeP)
    timeP = timeC
    
    cv2.putText(frame, str(int(fps)), (5,50), cv2.FONT_HERSHEY_DUPLEX, 3, (0,255,0), 3)
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cv2.destroyAllWindows()